# TAPOSITION


Francesco Di Bella, Progetto d'esame TAP 2023/2024

## Overview
> L'idea per TAPOSITION nasce da una esperienza di tirocinio, in cui andavano mappati dati da standard per l'e-learning (SCORM e XAPI ndr) ad uno standard europeo (NGSI-LD) che permette di ottenere l'evoluzione temporale di entità. Da qui l'idea che l'**evoluzione temporale** di un utente di un videogioco ( VR/AR o esperienza formativa/lavorativa) potesse essere rappresentata tramite la sua **posizione** nello **spazio** .

<a href="https://imgflip.com/i/8ymdfb"><img src="https://i.imgflip.com/8ymdfb.jpg" title="made at imgflip.com"/></a><div><a href="https://imgflip.com/memegenerator">from Imgflip Meme Generator</a></div>

## Caso d'uso
> Immaginiamo un software di gestione degli utenti di **esperienze immersive** in ambito **LET (Learning, Education, Training)**. 
Gli insegnanti potrebbe aver bisogno di un tool che permetta loro di visualizzare il percorso effettuato dall'utente e di ottenere (tramite un modello opportunamente trainato) in tempo reale delle prediction sulla prossima posizione

<img src="./images/useCase.png" alt="Il caso d'uso" />


<img src="https://miro.medium.com/v2/resize:fit:4800/format:webp/1*hSUKYQ85Am4SgphmV4dqYg.jpeg">

## Approccio ad una soluzione
>Tramite lo **stack ELK** e altre **tecnologie** a coadiuvare, è possibile **tenere traccia in tempo reale** dello stream di dati che l'**applicativo in uso** dall'utente **invia a Logstash** e che viene poi elaborato dalla pipeline. **Spark si occupa di elaborare le posizioni** e di restituire la **predicted position** 

<a href="https://imgflip.com/i/8ynovd"><img src="https://i.imgflip.com/8ynovd.jpg" title="made at imgflip.com"/></a><div><a href="https://imgflip.com/memegenerator">from Imgflip Meme Generator</a></div>

## Implementazioni future e idee
> - Includere altre mappe nel geoserver
> - Implementare le visualizzazioni Kibana in una pagina web esterna
> - Tracking di altre proprietà di utente e oggetti nella scena 3D
> - Integrare un chatbot o AI per comprendere e interrogare i dati da interfaccia grafica

## Tecnologie usate

<img src="./images/tecs.png"/>

<a href="https://imgflip.com/i/8ynodp"><img src="https://i.imgflip.com/8ynodp.jpg" title="made at imgflip.com"/></a><div><a href="https://imgflip.com/memegenerator">from Imgflip Meme Generator</a></div>

<a href="https://imgflip.com/i/8yo0se"><img src="https://i.imgflip.com/8yo0se.jpg" title="made at imgflip.com"/></a><div><a href="https://imgflip.com/memegenerator">from Imgflip Meme Generator</a></div>

## Data Producer: Applicazione 3D
> In questa pipeline il producer dei dati sarà un **qualsiasi applicativo 3D** (o non) capace di fare richieste HTTP a Logstash. Le tecnologie, **gli engine di gioco** (Unity, Unreal Engine, Godot ecc.) **non hanno nessun impatto sulla compatibilità** con gli altri componenti dell'architettura. Per il progetto è stato creato un applicativo di demo usando il framework JavaScript **Playcanvas**, infatti è possibile eseguire la scena 3D direttamente nel browser.

<image src="./images/3dgame.gif" style="width: 50%" />

<a href="https://imgflip.com/i/8yq4h7"><img src="https://i.imgflip.com/8yq4h7.jpg" title="made at imgflip.com"/></a><div><a href="https://imgflip.com/memegenerator">from Imgflip Meme Generator</a></div>

## Il data model per interagire con Logstash

<image src="./images/logastashDatamodel.png" />

Questa struttura del body permette di **ricostruire l'informazione** relativa ad una **particolare proprietà** di **qualsiasi oggetto in scena**, in questo caso si è ristretto il campo al giocatore.

<a href="https://imgflip.com/i/8yxyi6"><img src="https://i.imgflip.com/8yxyi6.jpg" title="made at imgflip.com"/></a><div><a href="https://imgflip.com/memegenerator">from Imgflip Meme Generator</a></div>

Di seguito alcuni frammenti del file **.conf** di Logstash

In [ ]:
input {
  http {
    id => "taposition_logstash"
    port => 9190
    response_headers => {
          "Access-Control-Allow-Origin" => "*"
          "Content-Type" => "text/plain"
          "Access-Control-Allow-Headers" => "Origin, X-Requested-With, Content-Type, Accept"
    }
  }
}

Tramite il plug-in **"input"** è possibile permettere a Logstash di ascoltare sulla porta designata e captare i messaggi in ingresso

L'oggetto **"response_headers"** è necessario per questo progetto in quanto la scena 3D sviluppata in Playcanvas esige degli header di risposta specifici (CORS related) 

<img src="./images/tvw3y3jyfe351.webp" title="made at imgflip.com"/>

In [ ]:
ruby {
    code => '
        x = event.get("[location][lon]").to_f  # Convert x to a float
        z = event.get("[location][lat]").to_f   # Convert z to a float
        # Map x from (-10, 10) to (-30, 30)
        mapped_longitude = -x * 3
        # Map z from (-10, 10) to (30, -30)
        mapped_latitude = z * 3
        event.set("[location][lon]", mapped_longitude)
        event.set("[location][lat]", mapped_latitude)

        prev_x = event.get("[source][lon]").to_f  # Convert x to a float
        prev_z = event.get("[source][lat]").to_f   # Convert z to a float
        # Map x from (-10, 10) to (-30, 30)
        mapped_prev_longitude = -prev_x * 3
        # Map z from (-10, 10) to (30, -30)
        mapped_prev_latitude = prev_z * 3
        event.set("[source][lon]", mapped_prev_longitude.to_f)
        event.set("[source][lat]", mapped_prev_latitude.to_f)
    '
}

Tramite il plug-in **"ruby"** è possibile **eseguire del codice** (in Ruby), in questo caso sono stati eseguiti **calcoli per mappare** le posizioni da uno **spazio Euclideo ad un piano Geodetico** (detto datum), dove si distinguono longitudine e latitudine (ed in aggiunta anche altitudine). 

La conversione è lineare in quanto il terreno della scena 3D è 3 volte più piccolo dello spazio che occuperà la mappa proiettata sulla Terra (in termini di estensione e metri quadri).

Logstash invia i dati a Kafka (creando un topic chiamato "positions")

## Spark

Spark legge lo stream di dati dal topic "positions" di Kafka. Tramite la source (Logstash rinomina alcuni campi che abbiamo visto in precedenza) e tramite due modelli (uno per latitude e uno per longitude) permette di restiture la predicted_position tramite LinearRegression.

In [ ]:
def model_trainer(file_path):
    # Define schema for the data
    schema = StructType([
        StructField("lat", DoubleType(), True),
        StructField("lon", DoubleType(), True),
        StructField("next_lat", DoubleType(), True),
        StructField("next_lon", DoubleType(), True)
    ])

    # Load data
    data = spark.read.format("csv").options(header='true', inferschema='true', delimiter=",").load(file_path)

    # Assemble features
    assembler = VectorAssembler(inputCols=["lat", "lon"], outputCol="features")
    data = assembler.transform(data)

    # Split the data into training and testing sets
    train_data, test_data = data.randomSplit([0.8, 0.2], seed=12345)

    # Train model for latitude prediction
    lr_lat = LinearRegression(featuresCol="features", labelCol="next_lat")
    pipeline_lat = Pipeline(stages=[lr_lat])
    model_lat = pipeline_lat.fit(train_data)

    # Train model for longitude prediction
    lr_lon = LinearRegression(featuresCol="features", labelCol="next_lon")
    pipeline_lon = Pipeline(stages=[lr_lon])
    model_lon = pipeline_lon.fit(train_data)

    # Return the trained models and test data
    return model_lat, model_lon, test_data

Questa è la funzione principale che permette di trainare i modelli

## Kibana e le visualizzazioni
Kibana permette di creare delle dashboard complete, nel mio caso ho optato per la visualizzazione della mappa (custom tramite la connessione ad un GeoServer) e alcuni box contenenti informazioni generiche e filtri per gli utenti.

<img src="./images/kibana1.png"/>

<img src="./images/kibana2.png"/>